<a href="https://colab.research.google.com/github/bankvis7/cp-axtra-training-lab-3/blob/main/lab4_qlora_finetuning_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab การ Fine-tuning ด้วย QLoRA โดยใช้ Unsloth

<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a>
</div>

เนื้อหา:

1. การเตรียมและจัดรูปแบบชุดข้อมูลสำหรับการ fine-tuning แบบ instruction
2. การ Fine-tune โมเดล Gemma 2 2B บนชุดข้อมูล instruction-following
3. กำหนดค่าพารามิเตอร์ LoRA (rank, alpha, dropout, target modules)
4. การวิเคราะห์และเปรียบเทียบประสิทธิภาพ base model กับโมเดลที่ fine-tune แล้ว
5. การบันทึกโมเดลที่ fine-tune แล้วเพื่อการ Deployment

---

## QLoRA (Quantized Low-Rank Adaptation)

QLoRA เป็นการรวมเทคนิค:

1. **Quantization** ของ base model ด้วย NF4 เพื่อประหยัดหน่วยความจำ
2. **LoRA** สร้าง Low-Rank Adapter แทนการสร้างเมทริกซ์ขนาดใหญ่

### LoRA ทำงานอย่างไร

เราจะไม่แก้ weight ของ base model แต่จะเพิ่ม adapter เพื่อปรับแก้ผลลัพธ์ให้ถูกต้องแทน:

$$W_{\text{new}} = W_{\text{frozen}} + \frac{\alpha}{r} \; W_A W_B $$

โดยที่:
- $W_{\text{new}}$ คือ น้ำหนักของ base model ที่ถูก quantized เป็น 4-bit
- $W_A, W_B$ คือ เมทริกซ์ของ adapter
- $r$ คือ rank ของ adapter (rank r << hidden_dim)
- $\alpha$ คือ scaling factor เพื่อควบคุมผลกระทบของ adapter

----

## Hugging Face Token

ในการเข้าถึงโมเดล Gemma จะต้องยืนยันตัวตนกับ Hugging Face โดยใช้ read access token จาก [Hugging Face](https://huggingface.co/settings/tokens) และเพิ่มลงใน Colab secrets manager ให้ตั้งชื่อ secret ว่า `HF_TOKEN`

หรือ Run Cell ต่อไปนี้เพื่อ Login ผ่าน GUI

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

* * *

## T4 GPU Runtime

แลปนี้จำเป็นต้องใช้ GPU ในการประมวลผล ให้เปิดใช้งาน T4 GPU runtime โดยไปที่ "**Runtime/Connect (มุมบนขวา)**" -> "**Change runtime type**" และเลือก "**T4 GPU**" ภายใต้ "**Hardware accelerator**"

ตรวจสอบการเชื่อมต่อกับ GPU ด้วยคำสั่งต่อไปนี้:

In [ ]:
!nvidia-smi

* * *
## ส่วนที่ 1: การติดตั้ง Unsloth Library

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !uv pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !uv pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !uv pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !uv pip install --no-deps unsloth
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

## ส่วนที่ 2: โหลด Base Model แบบ 4-bit Quantization

ทำการโหลด Gemma2 2B แบบ 4-bit precision เพื่อเป็น Base Model ของ QLoRA

In [ ]:
MODEL_NAME = "unsloth/gemma-2-2b-it"
MAX_SEQ_LENGTH = 2048

In [ ]:
from unsloth import FastModel
import torch

# Clear GPU memory
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

print("\n" + "="*60)
print("LOADING BASE MODEL WITH 4-BIT QUANTIZATION")
print("="*60)

model, tokenizer = FastModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = True,  # QLoRA uses 4-bit quantization
    dtype = None,  # Auto-detect dtype
)

# Track initial memory
base_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\nBase model loaded: {base_memory} GB")
print(f"Model dtype: {model.dtype}")

## ส่วนที่ 3: กำหนดค่า LoRA Adapters

ทำการเพิ่ม LoRA adapters ให้กับโมเดล ส่วนนี้คือพารามิเตอร์เพียงส่วนเดียวที่จะถูก fine-tune

พารามิเตอร์ LoRA ที่สำคัญ
- r (rank)
- lora_alpha
- lora_dropout
- target_modules

รายละเอียดเกี่ยวกับพารามิเตอร์เหล่านี้สามารถดูได้ที่ [Unsloth Documentation](https://docs.unsloth.ai/get-started/fine-tuning-llms-guide/lora-hyperparameters-guide#hyperparameters-and-recommendations)


In [ ]:
print("\n" + "="*60)
print("ADDING LoRA ADAPTERS")
print("="*60)

model = FastModel.get_peft_model(
    model,
    r = 16,  # LoRA rank
    lora_alpha = 16,  # LoRA scaling
    lora_dropout = 0,  # No dropout (Unsloth optimized)
    
    # Target modules: attention layers
    target_modules = [
        "q_proj",  # Query projection
        "k_proj",  # Key projection
        "v_proj",  # Value projection
        "o_proj",  # Output projection
        "gate_proj",  # FFN gate
        "up_proj",  # FFN up
        "down_proj",  # FFN down
    ],
    
    bias = "none",  # Don't train bias terms
    use_gradient_checkpointing = "unsloth",  # Memory efficient
    random_state = 3407,
)

# Check trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
trainable_percent = 100 * trainable_params / all_params

print(f"\nTrainable parameters: {trainable_params:,}")
print(f"Total parameters: {all_params:,}")
print(f"Percentage trainable: {trainable_percent:.2f}%")

# Memory after adding LoRA
lora_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\nMemory with LoRA adapters: {lora_memory} GB")
print(f"Additional memory for LoRA: {lora_memory - base_memory:.3f} GB")

## ส่วนที่ 4: เตรียมชุดข้อมูล

ในแลปนี้ เราจะใช้ชุดข้อมูล [**Alpaca**](https://huggingface.co/datasets/yahma/alpaca-cleaned) เป็นตัวอย่างแบบ instruction-following จำนวน 52K รายการ

โดยรูปแบบของแต่ละรายการ ประกอบด้วย:
- **instruction**: คำสั่งหรือพรอมต์
- **input**: บริบทหรืออินพุตเสริม (ถ้ามี)
- **output**: ผลลัพธ์ที่ต้องการ

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

# Load dataset
print("\n" + "="*60)
print("LOADING ALPACA DATASET")
print("="*60)

dataset = load_dataset("yahma/alpaca-cleaned", split="train")

print(f"\nDataset size: {len(dataset):,} examples")
print("\nExample from dataset:")
print("="*60)
print(f"Instruction: {dataset[0]['instruction']}")
print(f"Input: {dataset[0]['input']}")
print(f"Output: {dataset[0]['output']}")
print("="*60)

### ตั้งค่า Chat Template

กำหนด chat template ของ Gemma เพื่อใช้จัดรูปแบบข้อมูลให้ตรงกับที่โมเดลต้องการ

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma",  # Use Gemma format
)

print("Chat template configured!")

### จัดรูปแบบชุดข้อมูล

แปลงรูปแบบ Alpaca เป็นรูปแบบ chat ของ Gemma

In [ ]:
def format_alpaca_to_chat(example):
    """Convert Alpaca format to chat format"""
    
    # Combine instruction and input
    if example['input'].strip():
        user_message = f"{example['instruction']}\n\n{example['input']}"
    else:
        user_message = example['instruction']
    
    # Create chat messages
    messages = [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": example['output']},
    ]
    
    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = False,
    )
    
    return {"text": text}

# Apply formatting
print("\nFormatting dataset...")
dataset = dataset.map(format_alpaca_to_chat)

# Show formatted example
print("\nFormatted example:")
print("="*60)
print(dataset[0]['text'][:500] + "...")
print("="*60)

### สร้างชุดข้อมูล Train/Test

เราจะใช้ 1000 ตัวอย่างสำหรับการ training (เพื่อให้การ training รวดเร็ว) และ 100 ตัวอย่างสำหรับการทดสอบ

In [ ]:
# For this lab, we'll use a subset for faster training
train_dataset = dataset.select(range(1000))  # First 1000 examples
test_dataset = dataset.select(range(1000, 1100))  # Next 100 examples

print(f"Training examples: {len(train_dataset):,}")
print(f"Test examples: {len(test_dataset):,}")

## ส่วนที่ 5: ทดสอบ Base Model (ก่อน Fine-tuning)

In [ ]:
# Test prompts
test_prompts = [
    "Write a haiku about artificial intelligence.",
    "Explain the concept of recursion in programming in simple terms.",
    "What are three tips for staying productive while working from home?",
]

In [ ]:
def test_model(model, tokenizer, prompt, max_tokens=200):
    """Helper function to test model with a prompt"""
    messages = [{"role": "user", "content": prompt}]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        tokenize = True,
        return_tensors = "pt",
        return_dict = True,
    ).to("cuda")
    
    outputs = model.generate(
        **inputs,
        max_new_tokens = max_tokens,
        temperature = 0.7,
        top_p = 0.9,
        do_sample = True,
    )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response


print("\n" + "="*60)
print("BASE MODEL RESPONSES (BEFORE FINE-TUNING)")
print("="*60)

base_responses = []
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n[Test {i}] Prompt: {prompt}")
    print("-"*60)
    response = test_model(model, tokenizer, prompt)
    base_responses.append(response)
    print(f"Response: {response}")
    print("-"*60)

## ส่วนที่ 6: กำหนดค่าการ Fine-tuning

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

print("\n" + "="*60)
print("CONFIGURING FINE-TUNING")
print("="*60)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    
    args = TrainingArguments(
        output_dir = "outputs",
        num_train_epochs = 1,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        learning_rate = 2e-4,
        warmup_steps = 5,
        optim = "adamw_8bit",
        logging_steps = 10,
        eval_strategy = "steps",
        eval_steps = 50,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        save_strategy = "epoch",
        save_total_limit = 1,
        seed = 3407,
        report_to = "none",  # Disable wandb, tensorboard, and all logging
    ),
)

print("\nFine-tuning configured! Ready to fine-tune.")

## ส่วนที่ 7: การ Fine-tune โมเดล

In [ ]:
import time

print("\n" + "="*60)
print("STARTING FINE-TUNING")
print("="*60)

torch.cuda.reset_peak_memory_stats()
start_time = time.time()

# Fine-tuning!
trainer_stats = trainer.train()

training_time = time.time() - start_time
peak_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)

print("\n" + "="*60)
print("FINE-TUNING COMPLETE!")
print("="*60)
print(f"Fine-tuning time: {training_time/60:.2f} minutes")
print(f"Peak GPU memory: {peak_memory} GB")
print(f"Final loss: {trainer_stats.training_loss:.4f}")
print("="*60)

## ส่วนที่ 8: ทดสอบโมเดลที่ Fine-tune แล้ว

In [ ]:
print("\n" + "="*60)
print("FINE-TUNED MODEL RESPONSES (AFTER FINE-TUNING)")
print("="*60)

finetuned_responses = []
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n[Test {i}] Prompt: {prompt}")
    print("-"*60)
    response = test_model(model, tokenizer, prompt)
    finetuned_responses.append(response)
    print(f"Response: {response}")
    print("-"*60)

### การเปรียบเทียบแบบเคียงข้าง

In [ ]:
print("\n" + "="*80)
print("BEFORE vs AFTER COMPARISON")
print("="*80)

for i, prompt in enumerate(test_prompts):
    print(f"\n[Test {i+1}] {prompt}")
    print("-"*80)
    print(f"BEFORE: {base_responses[i]}")
    print(f"\nAFTER:  {finetuned_responses[i]}")
    print("="*80)

## ส่วนที่ 9: การวิเคราะห์การใช้หน่วยความจำ

มาวิเคราะห์ว่า QLoRA ประหยัดหน่วยความจำได้มากแค่ไหนเมื่อเทียบกับการ fine-tuning แบบเต็ม

In [ ]:
import matplotlib.pyplot as plt

methods = ['QLoRA\n(This Lab)', 'Full Fine-tuning\n(BF16)', 'Full Model\n(BF16)']
memory_usage = [peak_memory, 16, 4]
colors = ['#2ecc71', '#e74c3c', '#f39c12']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(methods, memory_usage, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

for bar, mem in zip(bars, memory_usage):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{mem:.1f} GB',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('GPU Memory Usage (GB)', fontsize=12, fontweight='bold')
ax.set_title('Memory Efficiency: QLoRA vs Full Fine-tuning\nGemma-2 2B Model', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3, linestyle='--')

savings = ((16 - peak_memory) / 16) * 100
textstr = f'QLoRA Memory Savings:\n~{savings:.0f}% vs Full Fine-tuning'
props = dict(boxstyle='round', facecolor='lightgreen', alpha=0.8)
ax.text(0.98, 0.97, textstr, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', horizontalalignment='right', bbox=props)

plt.tight_layout()
plt.show()

print(f"\nQLoRA used only ~{(peak_memory/16)*100:.0f}% of the memory required for full fine-tuning!")

## ส่วนที่ 10: บันทึกโมเดล

บันทึกเฉพาะน้ำหนักของ LoRA Adapters ที่ Fine-tune แล้ว (~10-50 MB)

In [ ]:
import os

print("\n" + "="*60)
print("SAVING LoRA ADAPTERS")
print("="*60)

model.save_pretrained("gemma-2-2b-alpaca-lora")
tokenizer.save_pretrained("gemma-2-2b-alpaca-lora")

print("\nLoRA adapters saved to: gemma-2-2b-alpaca-lora/")

adapter_path = "gemma-2-2b-alpaca-lora/adapter_model.safetensors"
if os.path.exists(adapter_path):
    size_mb = os.path.getsize(adapter_path) / (1024 * 1024)
    print(f"Adapter size: {size_mb:.1f} MB")

----